## 프록시 디자인 패턴의 개요

프록시란 요청자와 공급자 사이의 중재자를 말한다.

디자인 패턴 관점에서 Proxy 클래스는 객체의 인터페이스 역할을 한다. 

객체는 네트워크 연결, 메모리, 파일에 저장된 객체 등 다양한 종류에 해당한다.

Proxy 클래스는 반환해 사용할 객체를 감싸는 포장지 또는 에이전트 객체다.

프록시 패턴의 주목적은 실제 객체에 접근할 수 있는 대리 객체나 껍데기를 제공하는 것이다.

프록시 패턴의 역할

* 복잡한 시스템을 간간하게 표현할 수 있다.
* 객체에 대한 보안을 제공한다.
* 일반적으로 클라이언트 객체에 직접 접근할 수 없다.
* 다른 서버에 존재하는 외부 객체에 대한 로컬 인터페이스를 제공한다. 분산 시스템 구조에서 클라이언트가 원격으로 특정 커맨트를 권한이 없어 수행하지 못하는 경우가 있다. 이런 경우 로컬 객체(프록시)에 요청을 보내고 프록시는 원격 서버에서 요청을 수행한다.
* 메모리 사용량이 높은 객체를 다루는 가벼운 핸들러 역할을 한다. 예를 들어 웹사이트 사용자의 프로필 사진은 목록에는 작은 이미지로 표시하고 세부 프로필에을 보여줄 때만 실제 사진을 로드하는 것이 효율적이다.

배우와 아이젠트의 관계를 통해 예제 구현

In [2]:
class Actor(object):
    def __init__(self):
        self.isBusy = False

    def occupied(self):
        self.isBusy = True
        print(type(self).__name__, '다른 영화 촬영 중')
    
    def available(self):
        self.isBusy = False
        print(type(self).__name__, '출연 가능')
    
    def getStatus(self):
        return self.isBusy

class Agent(object):
    def __init__(self):
        self.principal = None
    
    def work(self):
        self.actor = Actor()
        if self.actor.getStatus():
            self.actor.occupied()
        else:
            self.actor.available()
r = Agent()
r.work()

Actor 출연 가능


프록시 패턴의 역할은 다음과 같다.
    * 특정 객체의 대리 객체를 제공해 접근을 제어
    * 분산 구조를 지원하기 위한 레이어 또는 인터페이스를 제공
    * 의도하지 않은 케이스로부터 객체를 보호
    
## 프록시의 여러 유형들

### 가상 프록시

